In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv("../input/2021-travelers-modeling-competition/train_2021.csv")
train.head()

,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,liab_prct,channel,policy_report_filed_ind,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud
0,1,46,M,1.0,85,38301,1,1,Rent,80006,...,74,Broker,0,7530.940993,9.0,Compact,12885.45235,white,16161.33381,0
1,3,21,F,0.0,75,30445,0,1,Rent,15021,...,79,Online,0,2966.024895,4.0,Large,29429.45218,white,28691.96422,0
2,4,49,F,0.0,87,38923,0,1,Own,20158,...,0,Broker,0,6283.888333,3.0,Compact,21701.18195,white,22090.94758,1
3,5,58,F,1.0,58,40605,1,0,Own,15024,...,99,Broker,1,6169.747994,4.0,Medium,13198.27344,other,38329.58106,1
4,6,38,M,1.0,95,36380,1,0,Rent,50034,...,7,Broker,0,4541.387150,7.0,Medium,38060.21122,gray,25876.56319,0


In [3]:
train.columns

Index(['claim_number', 'age_of_driver', 'gender', 'marital_status',
       'safty_rating', 'annual_income', 'high_education_ind',
       'address_change_ind', 'living_status', 'zip_code', 'claim_date',
       'claim_day_of_week', 'accident_site', 'past_num_of_claims',
       'witness_present_ind', 'liab_prct', 'channel',
       'policy_report_filed_ind', 'claim_est_payout', 'age_of_vehicle',
       'vehicle_category', 'vehicle_price', 'vehicle_color', 'vehicle_weight',
       'fraud'],
      dtype='object')

## Information about data: 

- We have 24 exploratory variables and 1 dependent variable `Fraud`
- `claim_number` is the ID that goes into submission along with `Fraud` value.
- `'age_of_driver', 'gender', 'marital_status','safty_rating', 'annual_income', 'high_education_ind','address_change_ind', 'living_status', 'zip_code'` are variables w.r.t the driver
- `'past_num_of_claims', policy_report_filed_ind, channel` are driver's changing qualities
- `'claim_date', 'claim_day_of_week', 'accident_site','witness_present_ind', 'liab_prct', 'claim_est_payout'` are information w.r.t claim
- `'age_of_vehicle','vehicle_category', 'vehicle_price', 'vehicle_color', 'vehicle_weight'` are variables w.r.t vehicle

## Exploratory Analysis for Variables:

- We check for outliers, NA values, possibility of binning, univariate and bivariate relationships with `Fraud`.

### Dependent Variable: Fraud

In [4]:
print(f"number of NA's: {sum(train.fraud.isna())}")  ## checking number of NA's
print(f"number of nulls's: {sum(train.fraud.isnull())}")  ## checking number of null's

number of NA's: 0
number of nulls's: 0


In [5]:
print(f"All possiblee values of Fraud are {train.fraud.unique()}")  ## checking for unique values
print(f"Number of Fraudulent Cases are: {len(train.fraud[train.fraud == 1])}")
print(f"Number of Non-fraudulent Cases are: {len(train.fraud[train.fraud == 0])}")
print(f"Percentage of Fraudulent Cases are: {round(len(train.fraud[train.fraud == 1])/len(train.fraud), 4)}")
print(f"Odds Ratio: {round(len(train.fraud[train.fraud == 1])/len(train.fraud[train.fraud == 0]), 4)}")

All possiblee values of Fraud are [0 1]
Number of Fraudulent Cases are: 2816
Number of Non-fraudulent Cases are: 15182
Percentage of Fraudulent Cases are: 0.1565
Odds Ratio: 0.1855


### Independent Variables

### NA & Null Value Treatment

First we check for NA values and Null values for all variables:
 - We observe that marital status has 5 NA values 
 - Age of vehicle also has 8 NA values
 - Claim Estimated Payout has 17 NA values
 - witness_present_ind has 132 NA values
 
It is interesting to observe the percentage of Fraud amongst NA values before imputing especially because it could be explanatory especially from age of vehicle and witness_present_ind.

In [6]:
train.isna().sum(axis = 0)

claim_number                 0
age_of_driver                0
gender                       0
marital_status               5
safty_rating                 0
annual_income                0
high_education_ind           0
address_change_ind           0
living_status                0
zip_code                     0
claim_date                   0
claim_day_of_week            0
accident_site                0
past_num_of_claims           0
witness_present_ind        132
liab_prct                    0
channel                      0
policy_report_filed_ind      0
claim_est_payout            17
age_of_vehicle               8
vehicle_category             0
vehicle_price                0
vehicle_color                0
vehicle_weight               0
fraud                        0
dtype: int64

In [7]:
train.witness_present_ind.value_counts()

0.0    13709
1.0     4157
Name: witness_present_ind, dtype: int64

In [8]:
train.marital_status.value_counts()

1.0    12824
0.0     5169
Name: marital_status, dtype: int64

In [9]:
train.age_of_vehicle.groupby(train.fraud).median()

fraud
0    5.0
1    5.0
Name: age_of_vehicle, dtype: float64

In [10]:
train.marital_status.fillna(1.0, inplace = True)    ## fill the five values with majority
train.witness_present_ind.fillna(0.0, inplace = True)  ## fill with majority as well as witness not present when na
train.age_of_vehicle.fillna(5, inplace = True)

In [11]:
## fitting regression of claim_est_payout using vehicle_price and `age_of_vehicle`
import sklearn
from sklearn import linear_model

imput_mod1 = linear_model.LinearRegression()
imput_mod1.fit(train.loc[train.claim_est_payout.isna() == False, ['vehicle_price', 'age_of_vehicle']],
               train.claim_est_payout[train.claim_est_payout.isna() == False])

## predicted values for NA's
imput_output = pd.Series(imput_mod1.predict(train.loc[train.claim_est_payout.isna() == True,
                                                      ['vehicle_price', 'age_of_vehicle']]),
                         index = train.index[train.claim_est_payout.isna()==True])

In [12]:
train.claim_est_payout.fillna(imput_output, inplace = True)

Based on above analysis, NA values in `age_of_vehicle` looks informative whereas `witness_present_ind` is not informative. We have imputed NA values in `witness_present_ind` with zeros as it makes sense that witness was not present and also since that is the majority. For others that are categorical such as `marital status`, we impute with majority that is "Married". For `age_of_vehicle`, we impute using `5` which is the median amongst fraudulent and non-fraudulent cases. 

### Univariate Analysis:




In [13]:
import datetime
import seaborn as sns
train['claim_date_d'] = pd.Series([datetime.datetime.strptime(d, '%m/%d/%Y').strftime('%d')
                                   for d in train.claim_date], index = train.index)
train['claim_date_m'] = pd.Series([datetime.datetime.strptime(d, '%m/%d/%Y').strftime('%m')
                                   for d in train.claim_date], index = train.index)

In [14]:
driver_cont = ['age_of_driver', 'safty_rating', 'annual_income', 'zip_code']
driver_cat  = ['gender', 'marital_status', 'living_status', 'past_num_of_claims']
claim_cont  = ['claim_date_d', 'claim_date_m', 'liab_prct', 'claim_est_payout']
claim_cat   = ['claim_day_of_week', 'accident_site', 'witness_present_ind', ]
purchase_cat = ['channel', 'policy_report_filed_ind']
vehicle_cont = ['age_of_vehicle', 'vehicle_price', 'vehicle_weight']
vehicle_cat = ['vehicle_category', 'vehicle_color']

### variabes to explore further = date (break into month, date etc), zip_code

## continuous variables:
print(train.loc[:,driver_cont].describe())
print(train.loc[:,claim_cont].describe())
print(train.loc[:,vehicle_cont].describe())

       age_of_driver  safty_rating  annual_income      zip_code
count   17998.000000  17998.000000   17998.000000  17998.000000
mean       43.695466     73.562951   37367.655684  49875.595955
std        11.959819     15.346807    2957.297249  29214.655149
min        18.000000      1.000000      -1.000000      0.000000
25%        35.000000     65.000000   35554.000000  20111.000000
50%        43.000000     76.000000   37610.000000  50028.000000
75%        51.000000     85.000000   39318.000000  80038.000000
max       229.000000    100.000000   54333.000000  85083.000000
          liab_prct  claim_est_payout
count  17998.000000      17998.000000
mean      49.423269       4975.793719
std       33.678470       2214.659801
min        0.000000        282.639432
25%       17.000000       3339.205052
50%       50.000000       4671.827763
75%       81.000000       6254.708103
max      100.000000      17218.345010
       age_of_vehicle  vehicle_price  vehicle_weight
count    17998.000000   17998

In [15]:
## categorical variables:
print([train[x].value_counts(normalize = True) for x in driver_cat])
print([train[x].value_counts(normalize = True) for x in claim_cat])
print([train[x].value_counts(normalize = True) for x in purchase_cat])
print([train[x].value_counts(normalize = True) for x in vehicle_cat])

[M    0.523058
F    0.476942
Name: gender, dtype: float64, 1.0    0.712801
0.0    0.287199
Name: marital_status, dtype: float64, Own     0.553895
Rent    0.446105
Name: living_status, dtype: float64, 0    0.702856
1    0.170852
2    0.074564
3    0.032448
4    0.010390
5    0.007223
6    0.001667
Name: past_num_of_claims, dtype: float64]
[Saturday     0.146127
Sunday       0.146072
Thursday     0.142849
Monday       0.142794
Friday       0.141905
Tuesday      0.140571
Wednesday    0.139682
Name: claim_day_of_week, dtype: float64, Local          0.487165
Parking Lot    0.304034
Highway        0.208801
Name: accident_site, dtype: float64, 0.0    0.76903
1.0    0.23097
Name: witness_present_ind, dtype: float64]
[Broker    0.535226
Phone     0.320647
Online    0.144127
Name: channel, dtype: float64, 1    0.600678
0    0.399322
Name: policy_report_filed_ind, dtype: float64]
[Compact    0.336871
Large      0.332593
Medium     0.330537
Name: vehicle_category, dtype: float64, black     0.14557

### Bivariate Analysis:

#### Continuous Variables: 

In [16]:
train.loc[train.fraud == 1,['age_of_driver', 'safty_rating', 'annual_income']].describe()

,age_of_driver,safty_rating,annual_income
count,2816.000000,2816.000000,2816.000000
mean,42.049006,71.755327,37054.233310
std,11.034827,16.851229,2632.435946
min,19.000000,1.000000,-1.000000
25%,35.000000,63.000000,35549.750000
50%,42.000000,75.000000,37371.500000
75%,48.000000,84.000000,38697.000000
max,229.000000,100.000000,54333.000000


In [17]:
# sns.boxplot(x = train.fraud, y = train)

From the above analysis, it does seem like there are many outliers for "Drivers age" in both cases. We also observe from the box plot that fraudulent cases have a slightly younger population. We do think this variable may be important for final analysis.

## Encode all variables

- Graph the histogram/Boxplot and sometimes also bivariate with fraud
- Choose the corresponding quantiles as bins
- Create binned variables
- Check crosstab for binned variables to observe any crucial patterns

In [18]:
# sns.histplot(x = train.age_of_driver)
# sns.histplot(x = train.safty_rating)
# sns.histplot(x = train.safty_rating)
# sns.catplot(x = 'fraud', col = 'zip_code_1', data = train_encoded, kind = 'count')
# sns.boxplot(x = train.fraud, y = train.safty_rating.astype('int'))
# sns.catplot(x = 'fraud', col = 'past_num_of_claims', kind = 'count')
# sns.histplot(x = train.liab_prct)  ## lot of values are in top and bottom
# sns.boxplot(x = train.fraud, y = train.liab_prct)  ## shows that fraudulent are more in between with not high values or low values.
# sns.boxplot(x = train.fraud, y = train.claim_est_payout)  
# sns.histplot(x = train.claim_est_payout)
# sns.histplot(x = train.age_of_vehicle)  ## positively skewed (with still a bell) but tail is explanatory for fraud
# sns.histplot(x = train.vehicle_price)   ## positively skewed (tail is explanatory for non fraud)
# sns.boxplot(x = train.fraud, y = train.vehicle_price)  ## more tail points in non-fraud
# sns.boxplot(x = train.fraud, y = train.vehicle_weight)  ## seems less informative
# sns.histplot(x = train.vehicle_weight)  ## positively skewed and tail is possibly informative

In [19]:
pd.crosstab( train.past_num_of_claims, train.fraud, normalize = 'index') ## most interesting variable until now. (Use 0, [1, 2], [> 2])

fraud,0,1
past_num_of_claims,,
0,0.864743,0.135257
1,0.831545,0.168455
2,0.770492,0.229508
3,0.726027,0.273973
4,0.711230,0.288770
5,0.561538,0.438462
6,0.733333,0.266667


## Data Processing Step

- Cap 'age_of_driver' to 80
- Use a simple linear model for 'annual_income' based on 'age_of_driver' to address the high correlation
- Create longitude, latitude, and state variables based on the zip_code provided 0 will be NA for now (drop zip_code variable afterwards) 

In [20]:
# Capping 'age_of_driver' at 80
train['age_of_driver'] = train['age_of_driver'].clip(upper = 80)

# fitting regression of claim_est_payout using vehicle_price and `age_of_vehicle`

input_mod2 = linear_model.LinearRegression()
input_mod2.fit(train.loc[train.annual_income != -1, ['age_of_driver']],
               train.annual_income[train.annual_income != -1])

## predicted values for -1's
input_output2 = pd.Series(input_mod2.predict(train.loc[train.annual_income == -1,
                                                      ['age_of_driver']]),
                         index = train.index[train.annual_income == -1])

#replacing -1 values with predicted values
train.annual_income.replace(to_replace = ([-1] * len(input_output2)), 
                            value = input_output2, inplace = True)

### Creating State, Latitude, and Longitude Variables 

Install 'uszipcode' package before running this code.

In [21]:
# !pip3 install uszipcode

from uszipcode import SearchEngine

engine = SearchEngine()

def zip_lookup(row):
    if row.zip_code == 0:
        return pd.DataFrame(data = {
                "state" : [np.nan],
                "lat" : [np.nan],
                "lng" : [np.nan]
               })
    else:
        info = engine.by_zipcode(str(row.zip_code))
        # changing to pandas dataframe
        return pd.DataFrame(data = {
                "state" : [info.state],
                "lat" : [info.lat],
                "lng" : [info.lng]
               })
    
train["state"] = train.apply(lambda row: zip_lookup(row).at[0, 'state'], axis = 1)
train["latitude"] = train.apply(lambda row: zip_lookup(row).at[0, 'lat'], axis = 1)
train["longitude"] = train.apply(lambda row: zip_lookup(row).at[0, 'lng'], axis = 1)

Because NA values seem to be an issue later on, we use a back fill to replace NA values with the values of the previous row.

In [22]:
train["state"] = train.state.fillna(method = "bfill")
train["latitude"] = train.latitude.fillna(method = "bfill")
train["longitude"] = train.longitude.fillna(method = "bfill")

### Principal Component Analysis

In this section we will be computing the principal components for the continuous variables

In [23]:
## Standardizing the dataset

from sklearn.preprocessing import StandardScaler
train_cont=pd.DataFrame()
train_cont= train[['age_of_driver', 'safty_rating', 'annual_income', 'liab_prct', 'claim_est_payout',
                   'age_of_vehicle', 'vehicle_price', 'vehicle_weight','latitude','longitude']]
data_scaler = StandardScaler()
data_scaler.fit(train_cont)
scaled_data_frame = data_scaler.transform(train_cont)

In [24]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(scaled_data_frame)
x_pca = pca.transform(scaled_data_frame)
print(x_pca.shape)

(17998, 10)


In [25]:
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

Explained variation per principal component: [0.1986365  0.15208191 0.1022184  0.10179179 0.1002952  0.09907433
 0.09850156 0.09766225 0.04820551 0.00153256]


In [26]:
pca_fit = pd.DataFrame(data = x_pca, columns = ['pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10'])
pca_fit.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,0.417263,-0.972443,-0.753872,-2.176616,0.320760,-0.353586,0.953861,-0.252621,0.415337,-0.089204
1,-3.173104,-0.923487,0.832911,0.294499,0.611696,-0.192114,-0.871968,-0.525954,0.412220,0.417053
2,0.700300,-0.812256,-0.526333,0.227415,-0.796107,1.416344,-0.170146,1.018699,0.885163,-0.067770
3,1.728583,-1.006439,1.252082,0.102218,0.024733,0.255119,0.861574,-1.879591,0.360928,0.042400
4,-0.677120,-0.789103,-0.392781,-0.258006,1.173105,0.491318,-0.241652,2.002551,-0.925197,-0.079059


In [27]:
train['pc1']=pca_fit[['pc1']]
train['pc2']=pca_fit[['pc2']]
train['pc3']=pca_fit[['pc3']]
train['pc4']=pca_fit[['pc4']]
train['pc5']=pca_fit[['pc5']]
train['pc6']=pca_fit[['pc6']]
train['pc7']=pca_fit[['pc7']]
train['pc8']=pca_fit[['pc8']]
train['pc9']=pca_fit[['pc9']]
train['pc10']=pca_fit[['pc10']]

In [28]:
train.head()

,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,1,46,M,1.0,85,38301.0,1,1,Rent,80006,...,0.417263,-0.972443,-0.753872,-2.176616,0.320760,-0.353586,0.953861,-0.252621,0.415337,-0.089204
1,3,21,F,0.0,75,30445.0,0,1,Rent,15021,...,-3.173104,-0.923487,0.832911,0.294499,0.611696,-0.192114,-0.871968,-0.525954,0.412220,0.417053
2,4,49,F,0.0,87,38923.0,0,1,Own,20158,...,0.700300,-0.812256,-0.526333,0.227415,-0.796107,1.416344,-0.170146,1.018699,0.885163,-0.067770
3,5,58,F,1.0,58,40605.0,1,0,Own,15024,...,1.728583,-1.006439,1.252082,0.102218,0.024733,0.255119,0.861574,-1.879591,0.360928,0.042400
4,6,38,M,1.0,95,36380.0,1,0,Rent,50034,...,-0.677120,-0.789103,-0.392781,-0.258006,1.173105,0.491318,-0.241652,2.002551,-0.925197,-0.079059


In [29]:
scaled_data_frame = data_scaler.transform(train_cont)
df=pd.DataFrame(scaled_data_frame)
my_columns = ["age_of_driver", "safty_rating", "annual_income", "liab_prct", "claim_est_payout",
              "age_of_vehicle", "vehicle_price", "vehicle_weight","latitude","longitude"]
df.columns = my_columns
df['pc1']=pca_fit[['pc1']]
df['pc2']=pca_fit[['pc2']]
df['pc3']=pca_fit[['pc3']]
df['pc4']=pca_fit[['pc4']]
df['pc5']=pca_fit[['pc5']]
df['pc6']=pca_fit[['pc6']]
df['pc7']=pca_fit[['pc7']]
df['pc8']=pca_fit[['pc8']]
df['pc9']=pca_fit[['pc9']]
df['pc10']=pca_fit[['pc10']]

In [30]:
df.head()

,age_of_driver,safty_rating,annual_income,liab_prct,claim_est_payout,age_of_vehicle,vehicle_price,vehicle_weight,latitude,longitude,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,0.201190,0.745260,0.327790,0.729766,1.153775,1.768047,-0.851150,-0.570027,0.396550,0.952918,0.417263,-0.972443,-0.753872,-2.176616,0.320760,-0.353586,0.953861,-0.252621,0.415337,-0.089204
1,-1.935657,0.093641,-2.525551,0.878233,-0.907509,-0.446472,0.528885,0.469683,0.396550,0.952918,-3.173104,-0.923487,0.832911,0.294499,0.611696,-0.192114,-0.871968,-0.525954,0.412220,0.417053
2,0.457612,0.875584,0.553703,-1.467544,0.590669,-0.889376,-0.115777,-0.078026,-0.100558,1.169501,0.700300,-0.812256,-0.526333,0.227415,-0.796107,1.416344,-0.170146,1.018699,0.885163,-0.067770
3,1.226877,-1.014112,1.164615,1.472101,0.539129,-0.446472,-0.825056,1.269349,0.467565,0.997183,1.728583,-1.006439,1.252082,0.102218,0.024733,0.255119,0.861574,-1.879591,0.360928,0.042400
4,-0.482601,1.396880,-0.369928,-1.259690,-0.196156,0.882240,1.248829,0.236079,1.225063,-0.098380,-0.677120,-0.789103,-0.392781,-0.258006,1.173105,0.491318,-0.241652,2.002551,-0.925197,-0.079059


### Data Encoding

In [31]:
train_encoded = pd.DataFrame()

## modify all numerical to binned variables
train_encoded['claim_number'] = train['claim_number'].astype('category')
train_encoded['dr_age_bins'] = pd.cut(train.age_of_driver, 
                                      bins = train.age_of_driver.quantile([0, .05, .25, .75, .95, 1]), 
                                      include_lowest = True)
train_encoded['dr_safty_bins'] = pd.cut(train.safty_rating,
                                        bins = train.safty_rating.quantile([0, .1, .3, .8, .95, 1]), 
                                        include_lowest = True)
train_encoded['dr_annual_income'] = pd.cut(train.annual_income, 
                                           bins = train.annual_income.quantile([0, .1, .3, .8, .95, 1]),
                                           include_lowest = True)
train_encoded['zip_code_1'] = round(train.zip_code/10000, 0).astype('category')    # satisfy the same state
train_encoded['past_num_of_claims'] = pd.cut(train.past_num_of_claims,
                                             bins = train.past_num_of_claims.quantile([0, 0.8, 0.9, 1]),
                                             include_lowest = True) # ordinal
train_encoded['liab_prct'] = pd.cut(train.liab_prct, 
                                    bins = train.liab_prct.quantile([0, 0.25, 0.75, 1]),
                                    include_lowest = True)
train_encoded['claim_est_payout'] = pd.cut(train.claim_est_payout,
                                           bins = train.claim_est_payout.quantile([0, 0.05, .25, .75, .95, 1]),
                                           include_lowest = True)
train_encoded['age_of_vehicle'] = pd.cut(train.age_of_vehicle,
                                         bins = train.age_of_vehicle.quantile([0, .25, .75, .9, 1]),
                                         include_lowest = True)
train_encoded['vehicle_price'] = pd.cut(train.vehicle_price,
                                        bins = train.vehicle_price.quantile([0, .1, .25, .75, .9, 1]),
                                        include_lowest = True)
train_encoded['vehicle_weight'] = pd.cut(train.vehicle_weight,
                                         bins = train.vehicle_weight.quantile([0, .1, .25, .75, .9, 1]),
                                         include_lowest = True)

# not binned numeric
train_encoded['latitude'] = train['latitude']
train_encoded['longitude'] = train['longitude']


## derived variables as categorical
# train_encoded['claim_date_d'] = train['claim_date_d'].astype('category')
# train_encoded['claim_date_m'] = train['claim_date_m'].astype('category')

## already categorical variables
train_encoded['gender'] = train['gender']
train_encoded['marital_status'] = train['marital_status'].astype('category')
train_encoded['high_education_ind'] = train['high_education_ind'].astype('category')
train_encoded['address_change_ind'] = train['address_change_ind'].astype('category')
train_encoded['living_status'] = train['living_status']
train_encoded['claim_day_of_week'] = train['claim_day_of_week']
train_encoded['witness_present_ind'] = train['witness_present_ind'].astype('category')
train_encoded['channel'] = train['channel']
train_encoded['policy_report_filed_ind'] = train['policy_report_filed_ind'].astype('category')
train_encoded['vehicle_category'] = train['vehicle_category']
train_encoded['vehicle_color'] = train['vehicle_color']
train_encoded['state'] = train['state'].astype('category')
train_encoded['fraud'] = train['fraud'].astype('category')

In [32]:
#pd.crosstab(train_encoded.zip_code_1,
#            train_encoded.fraud, normalize = 'index')
#pd.crosstab(train_encoded.dr_age_bins, 
#            train_encoded.fraud, normalize = 'index')
#pd.crosstab(train_encoded.dr_safty_bins, 
#            train_encoded.fraud, normalize = 'index')  ## interesting since higher safty has more fraudulent behavior
#pd.crosstab(train_encoded.claim_date_d, 
#            train_encoded.fraud, normalize = 'index')   ## not so interesting
#pd.crosstab(train.claim_date_m,
#            train.fraud, normalize = 'index')   ## not so interesting as percentages seem similar
#pd.crosstab(train.claim_day_of_week,
#            train.fraud, normalize = 'index') ## not so interesting as well
#pd.crosstab(train.past_num_of_claims,
#            train.fraud, normalize = 'index') ## most interesting variable until now. (Use 0, [1, 2], [> 2])
#pd.crosstab(train.age_of_vehicle,
#            train.fraud, normalize = 'index')  ## more fraud on old cars

In [33]:
train_encoded

,claim_number,dr_age_bins,dr_safty_bins,dr_annual_income,zip_code_1,past_num_of_claims,liab_prct,claim_est_payout,age_of_vehicle,vehicle_price,...,address_change_ind,living_status,claim_day_of_week,witness_present_ind,channel,policy_report_filed_ind,vehicle_category,vehicle_color,state,fraud
0,1,"(35.0, 51.0]","(68.0, 87.0]","(36105.0, 39707.0]",8.0,"(-0.001, 1.0]","(17.0, 81.0]","(6254.708, 9085.816]","(8.0, 16.0]","(9854.12, 14279.575]",...,1,Rent,Friday,0.0,Broker,0,Compact,white,CO,0
1,3,"(17.999, 26.0]","(68.0, 87.0]","(28895.999, 33666.0]",2.0,"(-0.001, 1.0]","(17.0, 81.0]","(1954.214, 3339.205]","(3.0, 6.0]","(14279.575, 29562.233]",...,1,Rent,Thursday,1.0,Online,0,Large,white,PA,0
2,4,"(35.0, 51.0]","(68.0, 87.0]","(36105.0, 39707.0]",2.0,"(-0.001, 1.0]","(-0.001, 17.0]","(6254.708, 9085.816]","(-0.001, 3.0]","(14279.575, 29562.233]",...,1,Own,Tuesday,0.0,Broker,0,Compact,white,VA,1
3,5,"(51.0, 65.0]","(53.0, 68.0]","(39707.0, 41722.15]",2.0,"(2.0, 6.0]","(81.0, 100.0]","(3339.205, 6254.708]","(3.0, 6.0]","(9854.12, 14279.575]",...,0,Own,Thursday,0.0,Broker,1,Medium,other,PA,1
4,6,"(35.0, 51.0]","(93.0, 100.0]","(36105.0, 39707.0]",5.0,"(-0.001, 1.0]","(-0.001, 17.0]","(3339.205, 6254.708]","(6.0, 8.0]","(29562.233, 38895.897]",...,0,Rent,Tuesday,1.0,Broker,0,Medium,gray,IA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,29993,"(65.0, 80.0]","(87.0, 93.0]","(41722.15, 54333.0]",2.0,"(-0.001, 1.0]","(81.0, 100.0]","(3339.205, 6254.708]","(3.0, 6.0]","(29562.233, 38895.897]",...,0,Own,Saturday,0.0,Broker,0,Medium,black,PA,0
17994,29996,"(26.0, 35.0]","(0.999, 53.0]","(33666.0, 36105.0]",2.0,"(-0.001, 1.0]","(81.0, 100.0]","(3339.205, 6254.708]","(-0.001, 3.0]","(2457.328, 9854.12]",...,1,Own,Monday,0.0,Phone,1,Medium,blue,VA,1
17995,29997,"(26.0, 35.0]","(68.0, 87.0]","(28895.999, 33666.0]",2.0,"(1.0, 2.0]","(-0.001, 17.0]","(1954.214, 3339.205]","(3.0, 6.0]","(38895.897, 127063.506]",...,1,Rent,Saturday,0.0,Online,1,Medium,black,PA,0
17996,29999,"(51.0, 65.0]","(68.0, 87.0]","(36105.0, 39707.0]",2.0,"(-0.001, 1.0]","(17.0, 81.0]","(9085.816, 17218.345]","(-0.001, 3.0]","(14279.575, 29562.233]",...,0,Own,Friday,1.0,Broker,1,Compact,white,PA,0


In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

list_ordinal = ['dr_safty_bins', 'dr_annual_income', 'past_num_of_claims','liab_prct',
                'claim_est_payout', 'age_of_vehicle', 'vehicle_price', 'vehicle_weight']
list_other = ['dr_age_bins', 'zip_code_1', 'gender', 'marital_status', 'high_education_ind',
              'address_change_ind', 'living_status', 'claim_day_of_week', 
              'witness_present_ind', 'channel', 'policy_report_filed_ind', 'vehicle_category',
              'vehicle_color', 'state']

encoding = [('ord', OrdinalEncoder(), list_ordinal), ('cat', OneHotEncoder(), list_other)]
# encoding = [('ord', OneHotEncoder(), list_ordinal), ('cat', OneHotEncoder(), list_other)]
col_transform = ColumnTransformer(transformers=encoding)

X = train_encoded.iloc[:, 1:-1]
y = train_encoded.iloc[:,  -1]

X_1 = col_transform.fit_transform(X)

from sklearn.model_selection import train_test_split
X_in1, X_hold1, y_in, y_hold = train_test_split(X_1, y, test_size=0.3, random_state=341)

In [35]:
train_encoded.dr_age_bins.value_counts()
train_encoded.vehicle_price.value_counts()
train_encoded.claim_est_payout.value_counts()
train_encoded.dr_annual_income.value_counts()
#test_encoded.dr_age_bins.value_counts()

(36105.0, 39707.0]      8998
(33666.0, 36105.0]      3598
(39707.0, 41722.15]     2688
(28895.999, 33666.0]    1814
(41722.15, 54333.0]      900
Name: dr_annual_income, dtype: int64

### Test Data preprocessing

In [36]:
test = pd.read_csv("../input/2021-travelers-modeling-competition/test_2021.csv")
test.isna().sum(axis = 0)

claim_number                0
age_of_driver               0
gender                      0
marital_status              2
safty_rating                0
annual_income               0
high_education_ind          0
address_change_ind          0
living_status               0
zip_code                    0
claim_date                  0
claim_day_of_week           0
accident_site               0
past_num_of_claims          0
witness_present_ind        88
liab_prct                   0
channel                     0
policy_report_filed_ind     0
claim_est_payout           14
age_of_vehicle              3
vehicle_category            0
vehicle_price               0
vehicle_color               0
vehicle_weight              0
dtype: int64

In [37]:
test.age_of_vehicle.fillna(5, inplace = True)
test.claim_est_payout.fillna(pd.Series(imput_mod1.predict(test.loc[test.claim_est_payout.isna() == True,
                                                                   ['vehicle_price', 'age_of_vehicle']]),
                                       index = test.index[test.claim_est_payout.isna() == True]), inplace = True)
test.witness_present_ind.fillna(0.0, inplace = True)
test.marital_status.fillna(1.0, inplace = True)

In [38]:
x =  test.loc[test.claim_number.isin([7999, 9002, 4757, 15904, 16207, 24506]),:]
# x_encoded =  pd.cut(x.age_of_driver, bins = [train.age_of_driver.quantile([0, .05, .25, .75, .95]), np.inf],  include_lowest=True, right = False, )
# x_encoded

## Manual Encoding of more outliers (by bringing them to closeby intervals (should create better groups with np.inf)
# test.loc[test.claim_number == 7999, 'age_of_driver'] = 200
# test.loc[test.claim_number == 9002, 'age_of_driver'] = 200

# test.loc[test.claim_number == 15904, 'vehicle_price'] = 3000
# test.loc[test.claim_number == 24506, 'vehicle_price'] = 3000

# test.loc[test.claim_number == 4757, 'claim_est_payout'] = 17000
# test.loc[test.claim_number == 16207, 'claim_est_payout'] = 17000

# test.loc['claim_number' == 7999, 'dr_age_bins']


test['age_of_driver'] = test['age_of_driver'].replace([278,249],200)
test['vehicle_price'] = test['vehicle_price'].replace([2261.507903, 2441.288933], 3000)
test['claim_est_payout'] = test['claim_est_payout'].replace([18621.353670, 23903.366250], 17000)
test['annual_income'] = test['annual_income'].replace([56275, 55189], 54000)

In [39]:
test['claim_date_d'] = pd.Series([datetime.datetime.strptime(d, '%m/%d/%Y').strftime('%d')
                                   for d in test.claim_date], index = test.index)
test['claim_date_m'] = pd.Series([datetime.datetime.strptime(d, '%m/%d/%Y').strftime('%m')
                                   for d in test.claim_date], index = test.index)

In [40]:
# Capping 'age_of_driver' at 80
test['age_of_driver'] = test['age_of_driver'].clip(upper = 80)


## predicted values for -1's
input_output3 = pd.Series(input_mod2.predict(test.loc[test.annual_income == -1,
                                                      ['age_of_driver']]),
                         index = test.index[test.annual_income == -1])

#replacing -1 values with predicted values
test.annual_income.replace(to_replace = ([-1] * len(input_output3)), 
                            value = input_output3, inplace = True)

In [41]:
test["state"] = test.apply(lambda row: zip_lookup(row).at[0, 'state'], axis = 1)
test["latitude"] = test.apply(lambda row: zip_lookup(row).at[0, 'lat'], axis = 1)
test["longitude"] = test.apply(lambda row: zip_lookup(row).at[0, 'lng'], axis = 1)

test["state"] = test.state.fillna(method = "bfill")
test["latitude"] = test.latitude.fillna(method = "bfill")
test["longitude"] = test.longitude.fillna(method = "bfill")

In [42]:
## PCA

test_cont=pd.DataFrame()
test_cont= test[['age_of_driver', 'safty_rating', 'annual_income', 'liab_prct', 'claim_est_payout',
                   'age_of_vehicle', 'vehicle_price', 'vehicle_weight','latitude','longitude']]
data_scaler1 = StandardScaler()
data_scaler1.fit(test_cont)
scaled_df = data_scaler1.transform(test_cont)

pca = PCA()
pca.fit(scaled_data_frame)
x_pca1 = pca.transform(scaled_df)
print(x_pca1.shape)

pca_fit1 = pd.DataFrame(data = x_pca1, columns = ['pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10'])
test['pc1']=pca_fit1[['pc1']]
test['pc2']=pca_fit1[['pc2']]
test['pc3']=pca_fit1[['pc3']]
test['pc4']=pca_fit1[['pc4']]
test['pc5']=pca_fit1[['pc5']]
test['pc6']=pca_fit1[['pc6']]
test['pc7']=pca_fit1[['pc7']]
test['pc8']=pca_fit1[['pc8']]
test['pc9']=pca_fit1[['pc9']]
test['pc10']=pca_fit1[['pc10']]
test.head()

(12002, 10)


,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,2,39,M,1.0,73,36633.0,1,0,Own,50048,...,-0.459386,-0.483547,-0.588896,-0.052976,0.821705,-0.296208,0.949680,0.702794,-0.697235,-0.085879
1,9,56,M,1.0,63,40252.0,1,1,Own,85068,...,1.503705,-0.499993,1.053055,-0.463964,-0.460108,-0.877418,0.659204,1.218093,0.968999,0.017104
2,11,39,F,0.0,75,36634.0,1,1,Own,20115,...,-0.493075,-0.434923,1.903643,-1.163120,0.958425,-1.218868,-0.252783,0.906934,0.992219,-0.083864
3,13,33,M,1.0,72,34960.0,0,1,Own,50006,...,-1.250439,-0.794944,0.897071,0.031381,-1.243992,1.063004,0.366292,0.016259,-0.936197,-0.019048
4,14,24,F,1.0,70,31776.0,1,1,Rent,20122,...,-2.606097,0.507129,-0.115721,1.033888,-1.614097,1.167679,-0.022030,0.123717,-0.857984,0.251495


In [43]:
df1=pd.DataFrame(scaled_df)
my_columns = ["age_of_driver", "safty_rating", "annual_income", "liab_prct", "claim_est_payout",
              "age_of_vehicle", "vehicle_price", "vehicle_weight","latitude","longitude"]
df1.columns = my_columns
df1['pc1']=pca_fit1[['pc1']]
df1['pc2']=pca_fit1[['pc2']]
df1['pc3']=pca_fit1[['pc3']]
df1['pc4']=pca_fit1[['pc4']]
df1['pc5']=pca_fit1[['pc5']]
df1['pc6']=pca_fit1[['pc6']]
df1['pc7']=pca_fit1[['pc7']]
df1['pc8']=pca_fit1[['pc8']]
df1['pc9']=pca_fit1[['pc9']]
df1['pc10']=pca_fit1[['pc10']]

In [44]:
df1.head()

,age_of_driver,safty_rating,annual_income,liab_prct,claim_est_payout,age_of_vehicle,vehicle_price,vehicle_weight,latitude,longitude,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,-0.380895,-0.045711,-0.260846,-0.737650,0.080327,1.360935,0.118527,0.309288,0.830347,-0.172052,-0.459386,-0.483547,-0.588896,-0.052976,0.821705,-0.296208,0.949680,0.702794,-0.697235,-0.085879
1,1.072900,-0.695977,1.049225,0.006846,1.050867,0.462824,1.386249,-0.572681,-0.249136,1.143572,1.503705,-0.499993,1.053055,-0.463964,-0.460108,-0.877418,0.659204,1.218093,0.968999,0.017104
2,-0.380895,0.084342,-0.260484,1.436277,0.264950,0.911880,2.263104,-0.122617,-0.249136,1.143572,-0.493075,-0.434923,1.903643,-1.163120,0.958425,-1.218868,-0.252783,0.906934,0.992219,-0.083864
3,-0.893999,-0.110738,-0.866468,-0.142053,1.306808,-1.333398,0.043930,0.315369,1.230155,-0.084872,-1.250439,-0.794944,0.897071,0.031381,-1.243992,1.063004,0.366292,0.016259,-0.936197,-0.019048
4,-1.663656,-0.240791,-2.019070,-1.154567,0.540424,-1.782453,-0.387097,0.007299,0.198650,-1.010565,-2.606097,0.507129,-0.115721,1.033888,-1.614097,1.167679,-0.022030,0.123717,-0.857984,0.251495


### Test data encoding

In [45]:
test_encoded = pd.DataFrame()

## modify all numerical to binned variables
test_encoded['claim_number'] = test['claim_number'].astype('category')
test_encoded['dr_age_bins'] = pd.cut(test.age_of_driver, 
                                     bins = train.age_of_driver.quantile([0, .05, .25, .75, .95, 1]), 
                                     include_lowest = True)
test_encoded['dr_safty_bins'] = pd.cut(test.safty_rating,
                                       bins = train.safty_rating.quantile([0, .1, .3, .8, .95, 1]),
                                       include_lowest = True)
test_encoded['dr_annual_income'] = pd.cut(test.annual_income,
                                          bins = train.annual_income.quantile([0, .1, .3, .8, .95, 1]),
                                          include_lowest = True )

test_encoded['zip_code_1'] = round(test.zip_code/10000, 0).astype('category')    # satisfy the same state
test_encoded['past_num_of_claims'] = pd.cut(test.past_num_of_claims,
                                            bins = train.past_num_of_claims.quantile([0, 0.8, 0.9, 1]),
                                            include_lowest = True) # ordinal
test_encoded['liab_prct'] = pd.cut(test.liab_prct, 
                                   bins = train.liab_prct.quantile([0, 0.25, 0.75, 1]),
                                   include_lowest = True)
test_encoded['claim_est_payout'] = pd.cut(test.claim_est_payout,
                                          bins = train.claim_est_payout.quantile([0, 0.05, .25, .75, .95, 1]), 
                                          include_lowest = True)
test_encoded['age_of_vehicle'] = pd.cut(test.age_of_vehicle,
                                        bins = train.age_of_vehicle.quantile([0, .25, .75, .9, 1]), 
                                        include_lowest = True)
test_encoded['vehicle_price'] = pd.cut(test.vehicle_price,
                                       bins = train.vehicle_price.quantile([0, .1, .25, .75, .9, 1]),
                                       include_lowest = True)
test_encoded['vehicle_weight'] = pd.cut(test.vehicle_weight,
                                        bins = train.vehicle_weight.quantile([0, .1, .25, .75, .9, 1]), 
                                        include_lowest = True)

# not binned numeric
train_encoded['latitude'] = train['latitude']
train_encoded['longitude'] = train['longitude']

## derived variables as categorical
# train_encoded['claim_date_d'] = train['claim_date_d'].astype('category')
# train_encoded['claim_date_m'] = train['claim_date_m'].astype('category')

## already categorical variables
test_encoded['gender'] = test['gender']
test_encoded['marital_status'] = test['marital_status'].astype('category')
test_encoded['high_education_ind'] = test['high_education_ind'].astype('category')
test_encoded['address_change_ind'] = test['address_change_ind'].astype('category')
test_encoded['living_status'] = test['living_status']
test_encoded['claim_day_of_week'] = test['claim_day_of_week']
test_encoded['witness_present_ind'] = test['witness_present_ind'].astype('category')
test_encoded['channel'] = test['channel']
test_encoded['policy_report_filed_ind'] = test['policy_report_filed_ind'].astype('category')
test_encoded['vehicle_category'] = test['vehicle_category']
test_encoded['vehicle_color'] = test['vehicle_color']
test_encoded['state'] = test['state'].astype('category')
# test_encoded['fraud'] = test['fraud'].astype('category')

In [46]:
test_encoded.isna().sum(axis = 0)

# test_encoded.loc[test_encoded.dr_age_bins.isna() == True ,['claim_number', 'dr_age_bins']]  # claims 7999, 9002
# test_encoded.loc[test_encoded.claim_est_payout.isna() == True ,['claim_number', 'claim_est_payout']]  # claims 4757,16207
# test_encoded.loc[test_encoded.vehicle_price.isna() == True ,['claim_number', 'vehicle_price']]   # claims 15904, 24506
# test_encoded.loc[test_encoded.dr_annual_income.isna() == True ,['claim_number', 'dr_annual_income']]   # claims 15904, 24506

# test.loc[test.claim_number.isin([7999, 9002, 4757, 15904, 16207, 24506]),:]

claim_number               0
dr_age_bins                0
dr_safty_bins              0
dr_annual_income           0
zip_code_1                 0
past_num_of_claims         0
liab_prct                  0
claim_est_payout           0
age_of_vehicle             0
vehicle_price              0
vehicle_weight             0
gender                     0
marital_status             0
high_education_ind         0
address_change_ind         0
living_status              0
claim_day_of_week          0
witness_present_ind        0
channel                    0
policy_report_filed_ind    0
vehicle_category           0
vehicle_color              0
state                      0
dtype: int64

In [47]:
list_ordinal = ['dr_safty_bins', 'dr_annual_income', 'past_num_of_claims','liab_prct', 'claim_est_payout',
                'age_of_vehicle', 'vehicle_price', 'vehicle_weight']

list_other = ['dr_age_bins', 'zip_code_1', 'gender', 'marital_status', 'high_education_ind', 'address_change_ind', 
             'living_status', 'claim_day_of_week', 'witness_present_ind', 'channel', 'policy_report_filed_ind', 
             'vehicle_category', 'vehicle_color', 'state']

encoding = [('ord', OrdinalEncoder(), list_ordinal), ('cat', OneHotEncoder(), list_other)]
# encoding = [('ord', OneHotEncoder(), list_ordinal), ('cat', OneHotEncoder(), list_other)]
col_transform = ColumnTransformer(transformers=encoding)

test_X = test_encoded.iloc[:, 1:]
test_X_1 = col_transform.fit_transform(test_X)

## Model building

### XGBoost

In [62]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve
from sklearn.model_selection import GridSearchCV
import itertools

cv_params = {'max_depth': [1,2,3,4,5,6],
             'min_child_weight': [1,2,3,4]}    # parameters to be tries in the grid search
fix_params = {'learning_rate': 0.2, 'n_estimators': 100,
              'objective': 'binary:logistic'}   #other parameters, fixed for the moment 
csv = GridSearchCV(XGBClassifier(**fix_params), cv_params, scoring = 'f1', cv = 5)

In [63]:
# csv.fit(X_in1, y_in)
# csv.best_params_      (Ran it to obtain {'max_depth': 6, 'min_child_weight': 1})

{'max_depth': 6, 'min_child_weight': 1} are the optimal values obtained after cross validation grid search.

In [140]:
## https://www.kaggle.com/dstuerzer/optimization-of-xgboost
# cv_params = {'subsample': [0.8,0.9,1], 'max_delta_step': [0,1,2,4]}
fix_params = {'learning_rate': 0.2,
              'n_estimators': 100, 
              'objective': 'binary:logistic', 
              'eval_metric': 'map' ,'max_depth': 6, 'min_child_weight':1}

# csv2 = GridSearchCV(XGBClassifier(**fix_params), cv_params, scoring = 'f1', cv = 5)
# csv2.fit(X_in1, y_in)
# csv2.best_params_

# {'max_delta_step': 1, 'subsample': 0.8} is obtained

In [146]:
# cv_params = {'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]}
fix_params['max_delta_step'] = 1
fix_params['subsample'] = 0.8
fix_params['eval_metric'] = 'map'
fix_params['enable_categorical'] = True

# csv3 = GridSearchCV(XGBClassifier(**fix_params), cv_params, scoring = 'f1', cv = 5)
# csv3.fit(X_in1, y_in)
# csv3.best_params_

# {'learning_rate': 0.3}

In [147]:
fix_params['learning_rate'] = 0.3
params_final =  fix_params
print(params_final)

{'learning_rate': 0.3, 'n_estimators': 100, 'objective': 'binary:logistic', 'eval_metric': 'map', 'max_depth': 6, 'min_child_weight': 1, 'max_delta_step': 1, 'subsample': 0.8, 'enable_categorical': True}


In [153]:
import xgboost as xgb

xgdmat_train = xgb.DMatrix(X_train, y_train)
xgdmat_test = xgb.DMatrix(X_hold)
xgb_final = xgb.train(params_final, xgdmat_train, num_boost_round = 100)

y_pred = xgb_final.predict(xgdmat_test)

params = {'thresh': [0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18]}
thresh = 0.08

# y_final [y_pred > thresh] = 1
# y_final [y_pred <= thresh] = 0

from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve
# confusion_matrix(y_hold, y_pred)

# [f1_score(y_hold, (y_pred > x).astype(int)) for x in params['thresh']]
## observed maximum between 0.06 and 0.1
[f1_score(y_hold, (y_pred > x).astype(int)) for x in np.linspace(0.06, 0.1, 5)]  ## maximum obtained at 0.07 which we will use


[11:02:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "enable_categorical", "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[0.3077490774907749,
 0.315748031496063,
 0.31675720852486416,
 0.3174182139699381,
 0.3173121791880541]

### Support Vector Machine

In [53]:
#[[f1_score(y_hold, (model.predict_proba(X_hold.values)[:,1]>x).astype(int)),x] for x in np.linspace(0.1,0.5,40)]

[[0.17880184331797233, 0.1],
 [0.17880184331797233, 0.11025641025641027],
 [0.17880184331797233, 0.12051282051282051],
 [0.17880184331797233, 0.13076923076923078],
 [0.17880184331797233, 0.14102564102564102],
 [0.17880184331797233, 0.1512820512820513],
 [0.17880184331797233, 0.16153846153846155],
 [0.17880184331797233, 0.1717948717948718],
 [0.17880184331797233, 0.18205128205128207],
 [0.17880184331797233, 0.19230769230769232],
 [0.17880184331797233, 0.20256410256410257],
 [0.17880184331797233, 0.21282051282051284],
 [0.17880184331797233, 0.2230769230769231],
 [0.17880184331797233, 0.23333333333333334],
 [0.17880184331797233, 0.24358974358974358],
 [0.17880184331797233, 0.25384615384615383],
 [0.17880184331797233, 0.2641025641025641],
 [0.17880184331797233, 0.2743589743589744],
 [0.17880184331797233, 0.2846153846153846],
 [0.17880184331797233, 0.2948717948717949],
 [0.17880184331797233, 0.30512820512820515],
 [0.17880184331797233, 0.3153846153846154],
 [0.17880184331797233, 0.325641025

In [141]:
## Data preprocessing for training dataset

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train['gd'] = label_encoder.fit_transform(train.gender)
train['ls'] = label_encoder.fit_transform(train.living_status)
train['cd'] = label_encoder.fit_transform(train.claim_day_of_week)
train['ch'] = label_encoder.fit_transform(train.channel)
train['vc'] = label_encoder.fit_transform(train.vehicle_category)
train['vcol'] = label_encoder.fit_transform(train.vehicle_color)
train['st'] = label_encoder.fit_transform(train.state)
train['ast'] = label_encoder.fit_transform(train.accident_site)

X = train[['age_of_driver', 'safty_rating', 'annual_income', 'liab_prct', 'claim_est_payout',
           'age_of_vehicle', 'vehicle_price', 'vehicle_weight','latitude','longitude','pc1', 
           'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','gd','ls','cd','ch',
           'vc','vcol','st','marital_status','high_education_ind','address_change_ind',
           'witness_present_ind','policy_report_filed_ind','ast','past_num_of_claims']]
y = train['fraud'].values

from sklearn.model_selection import train_test_split
X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.3, random_state=341)

In [142]:
## Data preprocessing for test dataset

test['gd'] = label_encoder.fit_transform(test.gender)
test['ls'] = label_encoder.fit_transform(test.living_status)
test['cd'] = label_encoder.fit_transform(test.claim_day_of_week)
test['ch'] = label_encoder.fit_transform(test.channel)
test['vc'] = label_encoder.fit_transform(test.vehicle_category)
test['vcol'] = label_encoder.fit_transform(test.vehicle_color)
test['st'] = label_encoder.fit_transform(test.state)
test['ast'] = label_encoder.fit_transform(test.accident_site)

X_test = test[['age_of_driver', 'safty_rating', 'annual_income', 'liab_prct', 'claim_est_payout',
           'age_of_vehicle', 'vehicle_price', 'vehicle_weight','latitude','longitude',
           'pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','gd','ls','cd','ch',
           'vc','vcol','st','marital_status','high_education_ind','address_change_ind',
           'witness_present_ind','policy_report_filed_ind','ast','past_num_of_claims']]

In [48]:
## Data preprocessing for training dataset

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['gd'] = label_encoder.fit_transform(train.gender)
df['ls'] = label_encoder.fit_transform(train.living_status)
df['cd'] = label_encoder.fit_transform(train.claim_day_of_week)
df['ch'] = label_encoder.fit_transform(train.channel)
df['vc'] = label_encoder.fit_transform(train.vehicle_category)
df['vcol'] = label_encoder.fit_transform(train.vehicle_color)
df['st'] = label_encoder.fit_transform(train.state)
df['ast'] = label_encoder.fit_transform(train.accident_site)
df['marital_status'] = train.marital_status
df['high_education_ind'] = train.high_education_ind
df['address_change_ind'] = train.address_change_ind
df['witness_present_ind'] = train.witness_present_ind
df['policy_report_filed_ind'] = train.policy_report_filed_ind
df['past_num_of_claims'] = train.past_num_of_claims
df['claim_date_d'] = train.claim_date_d
df['claim_date_m'] = train.claim_date_m

y = train['fraud'].values


from sklearn.model_selection import train_test_split
X_train, X_hold, y_train, y_hold = train_test_split(df, y, test_size=0.3, random_state=341)

In [49]:
X_hold.head()

,age_of_driver,safty_rating,annual_income,liab_prct,claim_est_payout,age_of_vehicle,vehicle_price,vehicle_weight,latitude,longitude,...,vc,vcol,st,ast,marital_status,high_education_ind,address_change_ind,witness_present_ind,policy_report_filed_ind,past_num_of_claims
11207,0.799507,0.614936,0.838093,-1.170610,-0.639148,0.439336,-0.751617,1.561349,0.132215,-0.974989,...,0,0,1,2,1.0,1,1,0.0,1,2
8128,0.543085,0.028479,0.627434,-0.457969,0.607390,0.439336,-0.442376,-1.250414,0.889713,-0.080200,...,1,1,2,2,0.0,0,1,0.0,0,0
7550,0.286664,0.940746,0.397888,-1.319077,0.829502,-0.446472,0.404574,-0.401596,0.775299,-0.076247,...,0,0,2,0,0.0,1,1,0.0,0,0
5891,-1.679236,1.592366,-2.042125,-0.962757,-0.536943,-0.003568,-0.394852,0.109539,0.503073,0.965565,...,0,0,3,1,0.0,1,1,1.0,0,0
201,-1.850183,-0.492817,-2.354481,-0.012568,-0.948670,-0.446472,-0.674264,0.960251,0.376823,0.993231,...,0,1,3,1,0.0,1,1,0.0,0,0


In [49]:
## Data preprocessing for test dataset

df1['gd'] = label_encoder.fit_transform(test.gender)
df1['ls'] = label_encoder.fit_transform(test.living_status)
df1['cd'] = label_encoder.fit_transform(test.claim_day_of_week)
df1['ch'] = label_encoder.fit_transform(test.channel)
df1['vc'] = label_encoder.fit_transform(test.vehicle_category)
df1['vcol'] = label_encoder.fit_transform(test.vehicle_color)
df1['st'] = label_encoder.fit_transform(test.state)
df1['ast'] = label_encoder.fit_transform(test.accident_site)
df1['marital_status'] = train.marital_status
df1['high_education_ind'] = test.high_education_ind
df1['address_change_ind'] = test.address_change_ind
df1['witness_present_ind'] = test.witness_present_ind
df1['policy_report_filed_ind'] = test.policy_report_filed_ind
df1['past_num_of_claims'] = test.past_num_of_claims
df1['claim_date_d'] = test.claim_date_d
df1['claim_date_m'] = test.claim_date_m

In [70]:
from sklearn.svm import SVC 

svc=SVC(kernel='rbf',class_weight='balanced')
svc.fit(X_train, y_train)
predictions = svc.predict(df1)

In [71]:
pred=pd.DataFrame(predictions, index = test.claim_number)
pred.value_counts()

0    7247
1    4755
dtype: int64

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_hold, predictions))

              precision    recall  f1-score   support

           0       0.90      0.65      0.76      4563
           1       0.24      0.61      0.35       837

    accuracy                           0.65      5400
   macro avg       0.57      0.63      0.55      5400
weighted avg       0.80      0.65      0.69      5400



In [ ]:
from sklearn.metrics import f1_score

f1_score(y_hold, knn_yhat)

In [57]:
import os
os.chdir(r'../working')

pred.to_csv(r"submission_st_svm_nodm.csv")

from IPython.display import FileLink
FileLink(r'submission_st_svm_nodm.csv')

C:\Users\astro\travelers\kaggle\working\submission_st_svm_nodm.csv

In [50]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001,'scale','auto'], 
              'class_weight':['balanced']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,scoring = 'f1') 
  
# fitting the model for grid search 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 35 candidates, totalling 175 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1;, score=0.000 total time=   6.8s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1;, score=0.000 total time=   6.8s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1;, score=0.000 total time=   6.7s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1;, score=0.271 total time=   6.8s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1;, score=0.000 total time=   6.8s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=0.1;, score=0.348 total time=   7.8s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=0.1;, score=0.324 total time=   7.9s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=0.1;, score=0.310 total time=   7.9s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=0.1;, score=0.314 total time=   7.8s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=0.1;, score=0.312 total time=   7.9s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=0.01;, score=0.356 total time=   7.1s
[

[CV 1/5] END C=10, class_weight=balanced, gamma=scale;, score=0.350 total time=   7.6s
[CV 2/5] END C=10, class_weight=balanced, gamma=scale;, score=0.312 total time=   7.6s
[CV 3/5] END C=10, class_weight=balanced, gamma=scale;, score=0.318 total time=   7.5s
[CV 4/5] END C=10, class_weight=balanced, gamma=scale;, score=0.317 total time=   7.4s
[CV 5/5] END C=10, class_weight=balanced, gamma=scale;, score=0.304 total time=   7.5s
[CV 1/5] END C=10, class_weight=balanced, gamma=auto;, score=0.314 total time=   8.0s
[CV 2/5] END C=10, class_weight=balanced, gamma=auto;, score=0.258 total time=   8.0s
[CV 3/5] END C=10, class_weight=balanced, gamma=auto;, score=0.287 total time=   7.8s
[CV 4/5] END C=10, class_weight=balanced, gamma=auto;, score=0.278 total time=   7.9s
[CV 5/5] END C=10, class_weight=balanced, gamma=auto;, score=0.286 total time=   7.7s
[CV 1/5] END C=100, class_weight=balanced, gamma=1;, score=0.000 total time=  19.4s
[CV 2/5] END C=100, class_weight=balanced, gamma=1;

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale',
                                   'auto']},
             scoring='f1', verbose=3)

In [51]:
#### With claim_date and month 

In [51]:
print(grid.best_params_) 

{'C': 100, 'class_weight': 'balanced', 'gamma': 0.0001}


In [56]:
grid_predictions = grid.predict(df1)
pred=pd.DataFrame(grid_predictions, index = test.claim_number)
pred.value_counts()

0    7124
1    4878
dtype: int64

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_hold, grid_predictions))

              precision    recall  f1-score   support

           0       0.91      0.65      0.76      4563
           1       0.25      0.65      0.36       837

    accuracy                           0.65      5400
   macro avg       0.58      0.65      0.56      5400
weighted avg       0.81      0.65      0.70      5400



In [57]:
import os
os.chdir(r'../working')

pred.to_csv(r"submission_svm_gscv.csv")

In [ ]:
## Without claim date and month

In [52]:
print(grid.best_params_) 

{'C': 10, 'class_weight': 'balanced', 'gamma': 0.001}


In [55]:
grid_predictions = grid.predict(df1)
pred=pd.DataFrame(grid_predictions, index = test.claim_number)
pred.value_counts()

0    7095
1    4907
dtype: int64

In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_hold, grid_predictions))

              precision    recall  f1-score   support

           0       0.91      0.64      0.75      4563
           1       0.25      0.65      0.36       837

    accuracy                           0.64      5400
   macro avg       0.58      0.65      0.56      5400
weighted avg       0.81      0.64      0.69      5400



In [56]:
import os
os.chdir(r'../working')

pred.to_csv(r"submission_svm_gscv_nodm.csv")